In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns 

%matplotlib inline 

In [2]:
o_p_t=pd.read_csv("order_products__train.csv")
o_p_p=pd.read_csv("order_products__prior.csv")
orders=pd.read_csv("orders.csv")
products=pd.read_csv("products.csv")
aisles=pd.read_csv("aisles.csv")
departments=pd.read_csv("departments.csv")

In [3]:
aisles['aisle'] = aisles['aisle'].astype('category')
departments['department'] = departments['department'].astype('category')
orders['eval_set'] = orders['eval_set'].astype('category')
products['product_name'] = products['product_name'].astype('category')

### Create a DataFrame with the orders and the products that have been purchased on prior orders (op)

In [4]:
order_opp=orders.merge(o_p_p, on='order_id', how='inner')
order_opp.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,NaN,196,1,0
1,2539329,1,prior,1,2,8,NaN,14084,2,0
2,2539329,1,prior,1,2,8,NaN,12427,3,0
3,2539329,1,prior,1,2,8,NaN,26088,4,0
4,2539329,1,prior,1,2,8,NaN,26405,5,0


## Create product predictors

In [5]:

product_total_purcahse = order_opp.groupby('product_id')['order_id'].count().to_frame('p_total_purchases')
product_total_purcahse.head()

,p_total_purchases
product_id,
1,1852
2,90
3,277
4,329
5,15


In [6]:
product_total_purcahse= product_total_purcahse.reset_index()
product_total_purcahse.head()

,product_id,p_total_purchases
0,1,1852
1,2,90
2,3,277
3,4,329
4,5,15


In [7]:
prob_reorder = order_opp.groupby('product_id').filter(lambda x: x.shape[0] >260)
prob_reorder.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,NaN,196,1,0
1,2539329,1,prior,1,2,8,NaN,14084,2,0
2,2539329,1,prior,1,2,8,NaN,12427,3,0
3,2539329,1,prior,1,2,8,NaN,26088,4,0
4,2539329,1,prior,1,2,8,NaN,26405,5,0


In [8]:
prob_reorder =prob_reorder.groupby('product_id')['reordered'].mean().to_frame('p_reorder_ratio')
prob_reorder = prob_reorder.reset_index()
prob_reorder.head()

,product_id,p_reorder_ratio
0,1,0.613391
1,3,0.732852
2,4,0.446809
3,10,0.506998
4,23,0.448502


In [9]:
product_total_purcahse = product_total_purcahse.merge(prob_reorder, on='product_id', how='left')

product_total_purcahse.head()

,product_id,p_total_purchases,p_reorder_ratio
0,1,1852,0.613391
1,2,90,NaN
2,3,277,0.732852
3,4,329,0.446809
4,5,15,NaN


In [10]:
product_total_purcahse['p_reorder_ratio'] = product_total_purcahse['p_reorder_ratio'].fillna(value=0)
product_total_purcahse.head()

,product_id,p_total_purchases,p_reorder_ratio
0,1,1852,0.613391
1,2,90,0.000000
2,3,277,0.732852
3,4,329,0.446809
4,5,15,0.000000
